In [2]:
import rasterio

# Pfad zur TIFF-Datei
input_tif = r"C:\Users\Marike\Desktop\Hochschule Bremen\2. Semester\2.3 Projekt\Warmelyse\Georeferencing\DOP10_Mosaik.tif"

try:
    with rasterio.open(input_tif) as src:
        # Prüfen auf CRS (Koordinatensystem)
        if src.crs:
            print("Die Datei ist eine GeoTIFF!")
            print("Koordinatensystem (CRS):", src.crs)
            print("Bounds (Grenzen):", src.bounds)
        else:
            print("Die Datei ist keine GeoTIFF. Keine geographischen Informationen gefunden.")
except Exception as e:
    print("Fehler beim Lesen der Datei:", e)

Die Datei ist eine GeoTIFF!
Koordinatensystem (CRS): EPSG:25832
Bounds (Grenzen): BoundingBox(left=487752.9, bottom=5881608.2, right=490828.79999999976, top=5884036.699999999)


In [ ]:
import json
import os
from shapely.geometry import Polygon
from pyproj import Transformer
import re
from pyproj import CRS

def extract_filename(filename):
    # Extrahiert die Koordinaten aus dem Dateinamen
    match = re.search(r'tile_(\d+)', filename)  # Nur die "tile_xxxx.png" ID extrahieren
    if match:
        return f"tile_{match.group(1)}"
    return None

def find_matching_metadata(metadata, tile_id):
    # Sucht nach den Metadaten anhand der tile_id
    for feature in metadata['features']:
        if feature['properties']['tile_id'] == tile_id:  # Vergleiche mit tile_id
            return feature
    return None

def pixel_to_coord(px, py, bbox, img_width, img_height):
    # Konvertiert Pixelkoordinaten in Weltkoordinaten
    world_width = bbox[2] - bbox[0]
    world_height = bbox[3] - bbox[1]
    x = bbox[0] + (px / img_width) * world_width
    y = bbox[3] - (py / img_height) * world_height
    return x, y

def get_bbox_from_polygon(polygon_coords):
    """
    Berechnet die Bounding-Box aus Polygonkoordinaten.
    :param polygon_coords: Liste von Koordinaten des Polygons
    :return: [minX, minY, maxX, maxY]
    """
    x_coords = [coord[0] for coord in polygon_coords]
    y_coords = [coord[1] for coord in polygon_coords]
    return [min(x_coords), min(y_coords), max(x_coords), max(y_coords)]

def create_geojson(model_output_file, metadata_file):
    with open(model_output_file, 'r') as f:
        model_output = json.load(f)

    # Lade die GeoJSON-Metadaten
    with open(metadata_file, 'r') as f:
        metadata = json.load(f)

    features = []

    for image in model_output['images']:
        image_file = os.path.basename(image['file_name'])
        tile_id = extract_filename(image_file)
        if not tile_id:
            print(f"Konnte keine tile_id aus {image_file} extrahieren. Überspringe dieses Bild.")
            continue

        # Suche nach den passenden Metadaten
        tile_metadata = find_matching_metadata(metadata, tile_id)
        if not tile_metadata:
            print(f"Keine passenden Metadaten für {image_file} gefunden. Überspringe dieses Bild.")
            continue

        # Berechne die Bounding-Box
        polygon_coords = tile_metadata['geometry']['coordinates'][0]
        bbox = get_bbox_from_polygon(polygon_coords)
        crs = "EPSG:25832"  # Festgelegtes Koordinatensystem

        transformer = Transformer.from_crs(CRS.from_string(crs), CRS.from_epsg(25832), always_xy=True)

        annotations = [ann for ann in model_output['annotations'] if ann['image_id'] == image['id']]

        for ann in annotations:
            if ann['category_id'] == 0 and ann['segmentation']:
                # Konvertiere die Segmentation in Weltkoordinaten
                world_coords = [
                    pixel_to_coord(x, y, bbox, image['width'], image['height'])
                    for x, y in zip(ann['segmentation'][0][::2], ann['segmentation'][0][1::2])
                ]
                epsg25832_coords = [transformer.transform(x, y) for x, y in world_coords]

                feature = {
                    "type": "Feature",
                    "geometry": {
                        "type": "Polygon",
                        "coordinates": [epsg25832_coords]
                    },
                    "properties": {
                        "image_id": ann['image_id'],
                        "annotation_id": ann['id'],
                        "area": ann['area']
                    }
                }
                features.append(feature)

    geojson = {
        "type": "FeatureCollection",
        "crs": {
            "type": "name",
            "properties": {
                "name": "EPSG:25832"  # Setze das CRS des GeoJSON auf EPSG:25832
            }
        },
        "features": features
    }

    return geojson



# Verwendung der Funktion
model_output_file = r'C:\Users\Marike\Desktop\Hochschule Bremen\2. Semester\2.3 Projekt\Warmelyse\Georeferencing\320x320 Annotationen\train_dataset_trees\result.json'
metadata_file = r'C:\Users\Marike\Desktop\Hochschule Bremen\2. Semester\2.3 Projekt\Warmelyse\Georeferencing\tif_output_trees\tiles_metadata_from_geotiff.geojson'
geojson_result = create_geojson(model_output_file, metadata_file)

# Speichern der GeoJSON-Datei
with open('trees_georeferenced.geojson', 'w') as f:
    json.dump(geojson_result, f, indent=2)

